
В этой домашней работе вам предстоит составить csv или pandas таблицу, в которой будет хранится информация о широком или узком топике текста.

Мы будем работать с тем же датасетом, что и на последнем семинаре по gensim.

In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim  

/opt/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')

df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [4]:

data = df.content.values.tolist()

data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]


data = [re.sub('\s+', ' ', sent) for sent in data]


data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])


<>:3: DeprecationWarning: invalid escape sequence \S
<>:6: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \S
<>:6: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \S
<>:6: DeprecationWarning: invalid escape sequence \s
<ipython-input-4-eee093cf667c>:3: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-4-eee093cf667c>:6: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  

data_words = list(sent_to_words(data))

In [6]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


In [7]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [187]:
df['data_lemmatized'] = data_lemmatized

In [188]:
type(df['data_lemmatized'])

pandas.core.series.Series

In [189]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

In [190]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


In [191]:
mallet_path = '/Users/ico/Desktop/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    

In [192]:
def coherencevalues(dictionary, corpus, texts, limit, start=2, step=2):
    opt_list = []
    models = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        models.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        opt_list.append(coherencemodel.get_coherence())
    return models, opt_list

In [193]:
models, opt_list = coherencevalues(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=30, step=2)

In [194]:
#Оптимальное число, должно быть 21, но у меня все слетело, поэтому я взяла шаг больше
opt_lists = opt_list.index(max(opt_list))
opt_lists

10

In [195]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=opt_lists, id2word=id2word)

In [196]:
data_1 = []
for i in ldamallet.show_topics(formatted=False, num_topics=opt_lists):
    data_1.append(i[0])

In [197]:
topics = []
number = []
for i in range(len(ldamallet.show_topics(formatted=False, num_topics=opt_lists))):
    data_mallet_1 = []
    data_mallet_2 = []
    for k in ldamallet.show_topics(formatted=False, num_topics=opt_lists)[i][1]:
        data_mallet_1.append(k[0])
        data_mallet_2.append(k[1])
    topics.append(data_mallet_1)
    number.append(data_mallet_2)
    

Далее создадим словарь из слов и их весов, а потом словарь из номера и словаря слов и весов

In [198]:
x = 0
new_1 = []
while x < len(data_1):
    data_dict = dict(zip(topics[x],number[x]))
    new_1.append(data_dict)
    x += 1


In [199]:
number = dict(zip(data_1, data_dict))

In [200]:
number

{0: {'line': 0.022074376506468592,
  'file': 0.008620689655172414,
  'build': 0.008583796546805057,
  'current': 0.00817797235476413,
  'power': 0.008079590732451179,
  'point': 0.007981209110138226,
  'number': 0.007833636676668799,
  'case': 0.007624575729253775,
  'source': 0.007612278026464656,
  'read': 0.0075507895125190615},
 1: {'people': 0.011578316650218525,
  'question': 0.011551882137318482,
  'claim': 0.00933138305371493,
  'make': 0.009313760045114902,
  'word': 0.009296137036514874,
  'true': 0.008961299873114338,
  'thing': 0.008564782179613704,
  'reason': 0.008388552093613421,
  'exist': 0.007806992809812491,
  'argument': 0.007049203440011279},
 2: {'year': 0.026993127596749326,
  'good': 0.022681543808304373,
  'game': 0.020787060022472496,
  'team': 0.01778201677598056,
  'play': 0.016083514071441637,
  'player': 0.012294546499777887,
  'make': 0.011798061093835742,
  'win': 0.0116935378504795,
  'line': 0.01166740703964044,
  'time': 0.011092529201181112},
 3: {'p

In [202]:
def what_topic(text):
    d = {i: 0 for i in range(len(nn))}
    for word in text:
        k = 0
        for num in range(opt_lists):
            if word in list(number[num].keys()):
                k += number[num].get(word)
                d[num]=k
    return max(d, key=d.get)

In [203]:
df['topic'] = df['data_lemmatized'].apply(what_topic)

In [204]:
df['topic']

0        6
1        9
2        6
3        6
4        6
        ..
11309    6
11310    9
11311    6
11312    9
11313    9
Name: topic, Length: 11314, dtype: int64

In [205]:
df['data_lemmatized']

0        [where, thing, car, nntp_poste, host, park, li...
1        [poll, final, call, summary, final, call, cloc...
2        [engineering, computer, network, distribution_...
3        [division, line, host, write, write, article, ...
4        [question, distribution, article, write, clear...
                               ...                        
11309    [scan, city, reply, line, consultation, cheap,...
11310    [screen, medford, old, problem, screen, blank,...
11311    [este, mount, mail, group, line, instal, try, ...
11312    [line, article, write, boy, embarasse, trivial...
11313    [steal, organization, line, distribution_usa, ...
Name: data_lemmatized, Length: 11314, dtype: object

In [206]:
topic_11 = pd.DataFrame({'number':df['topic'].index, 'theme':df['topic'].values})

In [207]:
topic_11

,number,theme
0,0,6
1,1,9
2,2,6
3,3,6
4,4,6
...,...,...
11309,11309,6
11310,11310,9
11311,11311,6
11312,11312,9


In [208]:
lemmas = pd.DataFrame({'number':df['data_lemmatized'].index, 'theme':df['data_lemmatized'].values})

In [233]:
final_df = topic_11.merge(lemmas, on='number')

In [234]:
final_df

,number,theme_x,theme_y
0,0,6,"[where, thing, car, nntp_poste, host, park, li..."
1,1,9,"[poll, final, call, summary, final, call, cloc..."
2,2,6,"[engineering, computer, network, distribution_..."
3,3,6,"[division, line, host, write, write, article, ..."
4,4,6,"[question, distribution, article, write, clear..."
...,...,...,...
11309,11309,6,"[scan, city, reply, line, consultation, cheap,..."
11310,11310,9,"[screen, medford, old, problem, screen, blank,..."
11311,11311,6,"[este, mount, mail, group, line, instal, try, ..."
11312,11312,9,"[line, article, write, boy, embarasse, trivial..."


Напишем функцию для получения топиков и слов, которые к ним принадлежат 